In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
import scikeras
from scikeras.wrappers import KerasRegressor
from tensorflow import keras
from sklearn.ensemble import VotingRegressor, RandomForestRegressor

In [2]:
df_train = pd.read_csv("./train_transform.csv")

In [3]:
df_test = pd.read_csv("./test_transform.csv")

In [4]:
def encode_category_1(category_1):
    if category_1 == 'B': return 2
    if category_1 == 'K': return 1
    return 0

def encode_category_2(category_2):
    if category_2 == 'D': return 2
    if category_2 == 'I': return 1
    return 0

def encode_df(df):
    df['CATEGORY_1'] = df['CATEGORY_1'].apply(encode_category_1)
    df['CATEGORY_2'] = df['CATEGORY_2'].apply(encode_category_2)
    df = pd.get_dummies(df, columns = ['CATEGORY_1', 'CATEGORY_2'])
    return df

In [145]:
def encode_in(df):
    df['CATEGORY_1'] = df['CATEGORY_1'].apply(encode_category_1)
    df['CATEGORY_2'] = df['CATEGORY_2'].apply(encode_category_2)
    if(df['CATEGORY_1']==0):
        df['CATEGORY_1_0']=1
        df['CATEGORY_1_1']=0
        df['CATEGORY_1_2']=0
        
    return df

In [5]:
df_train = encode_df(df_train)
df_test = encode_df(df_test)

In [31]:
df_train

,Unnamed: 0,UNIQUE_IDENTIFIER,STATUS_CHECK,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,...,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2,CATEGORY_1_0,CATEGORY_1_1,CATEGORY_1_2,CATEGORY_2_0,CATEGORY_2_1,CATEGORY_2_2
0,0,98481267304,0,0.098334,0.007531,0.043399,0.000000,0.000000,0.000714,0.000286,...,0.000000,0.0,4.449287,101.369863,1,0,0,1,0,0
1,0,98481267698,1,31.392245,3.803991,25.940547,0.000000,0.866865,11.122807,0.050339,...,0.000000,0.0,7.182779,115.068493,1,0,0,0,1,0
2,0,98481269325,0,0.018567,0.001624,0.010514,0.000000,0.005791,0.000278,0.000111,...,0.000000,0.0,3.977371,24.657534,1,0,0,0,0,1
3,0,98481271512,0,0.747600,0.117320,0.025330,0.000000,0.240000,0.504000,0.020400,...,0.000000,0.0,3.988122,10.958904,1,0,0,1,0,0
4,0,98481273023,0,0.500000,0.080000,0.000000,0.000000,0.000000,0.500000,0.100000,...,0.000000,0.0,4.128875,46.575342,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96293,0,98527316109,0,1.267884,0.189529,0.589838,0.000000,0.578000,0.384167,0.035167,...,0.000000,0.0,3.876182,10.958904,0,0,1,1,0,0
96294,0,98527316131,0,47.909122,6.131328,40.061926,0.000000,1.735889,4.858611,0.119500,...,0.000222,0.0,5.516531,24.657534,1,0,0,0,1,0
96295,0,98527316133,0,1.115944,0.148308,0.669854,0.305824,0.038648,0.371466,0.063021,...,0.000000,0.0,5.268355,449.315068,0,0,1,0,1,0
96296,0,98527316143,0,0.381386,0.057377,0.361288,0.006591,0.013333,0.000167,0.000067,...,0.000000,0.0,4.086504,156.164384,0,0,1,0,0,1


In [6]:
X_train1 = df_train.drop(['Unnamed: 0','UNIQUE_IDENTIFIER','Y1','Y2', 'WINNINGS_2', 'DEPOSIT_2','WITHDRAW_NUMBER', 'PRACTICE_ENTRY','PRACTICE_WINNINGS','PRACTICE_ENTRY_NUMBER','PRACTICE_WINNINGS_NUMBER'],axis=1)
X_train2 = df_train.drop(['Unnamed: 0','UNIQUE_IDENTIFIER','Y1','Y2','WITHDRAW', 'WITHDRAW_NUMBER', 'PRACTICE_ENTRY','PRACTICE_WINNINGS','PRACTICE_ENTRY_NUMBER','PRACTICE_WINNINGS_NUMBER'],axis=1)

X_test1 = df_test.drop(['Unnamed: 0','UNIQUE_IDENTIFIER', 'WINNINGS_2', 'DEPOSIT_2','WITHDRAW_NUMBER', 'PRACTICE_ENTRY','PRACTICE_WINNINGS','PRACTICE_ENTRY_NUMBER','PRACTICE_WINNINGS_NUMBER'],axis=1)
X_test2 = df_test.drop(['Unnamed: 0','UNIQUE_IDENTIFIER','WITHDRAW', 'WITHDRAW_NUMBER', 'PRACTICE_ENTRY','PRACTICE_WINNINGS','PRACTICE_ENTRY_NUMBER','PRACTICE_WINNINGS_NUMBER'],axis=1)

y1_train = df_train['Y1']
y2_train = df_train['Y2']

In [7]:
def get_reg(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1))
    return model

In [8]:
ensemble_model_y1 = VotingRegressor([
    ('random-forest', RandomForestRegressor(n_estimators=500)),
     ('mlp',KerasRegressor(model=get_reg, loss="mse", metrics=[KerasRegressor.r_squared], hidden_layer_sizes=(100,),epochs=500,batch_size=200,dropout=0.5)),
    ('xgb',XGBRegressor(n_estimators=800, learning_rate=0.1, max_depth=5))
])

In [9]:
ensemble_model_y2 = VotingRegressor([
    ('random-forest', RandomForestRegressor(n_estimators=1000)),
     ('mlp',KerasRegressor(model=get_reg, loss="mse", metrics=[KerasRegressor.r_squared], hidden_layer_sizes=(100,),epochs=600,batch_size=50,dropout=0.5)),
    ('xgb',XGBRegressor(n_estimators=400, learning_rate=0.1, max_depth=6, gamma=0.1))
])

In [11]:
ensemble_model_y1.fit(X_train1, y1_train)

Epoch 1/500
482/482 [==============================] - 2s 2ms/step - loss: 65.1216 - r_squared: -0.4526
Epoch 2/500
482/482 [==============================] - 1s 2ms/step - loss: 55.9753 - r_squared: -0.1976
Epoch 3/500
482/482 [==============================] - 1s 2ms/step - loss: 53.4824 - r_squared: 0.0294
Epoch 4/500
482/482 [==============================] - 1s 2ms/step - loss: 52.3559 - r_squared: 0.0729
Epoch 5/500
482/482 [==============================] - 1s 2ms/step - loss: 51.1756 - r_squared: 0.0690
Epoch 6/500
482/482 [==============================] - 1s 1ms/step - loss: 52.0996 - r_squared: 0.0642
Epoch 7/500
482/482 [==============================] - 1s 1ms/step - loss: 50.9372 - r_squared: 0.1206
Epoch 8/500
482/482 [==============================] - 1s 1ms/step - loss: 50.7868 - r_squared: 0.1220
Epoch 9/500
482/482 [==============================] - 1s 1ms/step - loss: 49.9903 - r_squared: -0.0984
Epoch 10/500
482/482 [==============================] - 1s 2ms/step - 

482/482 [==============================] - 1s 2ms/step - loss: 48.6048 - r_squared: 0.0770
Epoch 79/500
482/482 [==============================] - 1s 2ms/step - loss: 47.9615 - r_squared: 0.1593
Epoch 80/500
482/482 [==============================] - 1s 2ms/step - loss: 48.0431 - r_squared: 0.1676
Epoch 81/500
482/482 [==============================] - 1s 2ms/step - loss: 47.8726 - r_squared: 0.1413
Epoch 82/500
482/482 [==============================] - 1s 2ms/step - loss: 47.4798 - r_squared: 0.0942
Epoch 83/500
482/482 [==============================] - 1s 2ms/step - loss: 47.4602 - r_squared: 0.1499
Epoch 84/500
482/482 [==============================] - 1s 2ms/step - loss: 47.7533 - r_squared: 0.1370
Epoch 85/500
482/482 [==============================] - 1s 2ms/step - loss: 49.2754 - r_squared: 0.0583
Epoch 86/500
482/482 [==============================] - 1s 2ms/step - loss: 48.8745 - r_squared: 0.1606
Epoch 87/500
482/482 [==============================] - 1s 1ms/step - loss: 4

482/482 [==============================] - 1s 2ms/step - loss: 46.7685 - r_squared: 0.1836
Epoch 157/500
482/482 [==============================] - 1s 2ms/step - loss: 47.4666 - r_squared: 0.1233
Epoch 158/500
482/482 [==============================] - 1s 2ms/step - loss: 48.4224 - r_squared: 0.1608
Epoch 159/500
482/482 [==============================] - 1s 2ms/step - loss: 48.3234 - r_squared: 0.1891
Epoch 160/500
482/482 [==============================] - 1s 2ms/step - loss: 47.3462 - r_squared: 0.1254
Epoch 161/500
482/482 [==============================] - 1s 2ms/step - loss: 48.5098 - r_squared: 0.1812
Epoch 162/500
482/482 [==============================] - 1s 2ms/step - loss: 45.8280 - r_squared: 0.1100
Epoch 163/500
482/482 [==============================] - 1s 2ms/step - loss: 46.8098 - r_squared: 0.1399
Epoch 164/500
482/482 [==============================] - 1s 2ms/step - loss: 48.1974 - r_squared: 0.1716
Epoch 165/500
482/482 [==============================] - 1s 1ms/step 

482/482 [==============================] - 1s 2ms/step - loss: 47.5987 - r_squared: 0.0539
Epoch 235/500
482/482 [==============================] - 1s 2ms/step - loss: 45.7962 - r_squared: 0.1661
Epoch 236/500
482/482 [==============================] - 1s 2ms/step - loss: 47.6731 - r_squared: 0.1203
Epoch 237/500
482/482 [==============================] - 1s 1ms/step - loss: 48.3025 - r_squared: 0.1443
Epoch 238/500
482/482 [==============================] - 1s 2ms/step - loss: 47.5685 - r_squared: 0.1215
Epoch 239/500
482/482 [==============================] - 1s 2ms/step - loss: 45.1765 - r_squared: 0.1964
Epoch 240/500
482/482 [==============================] - 1s 2ms/step - loss: 48.4560 - r_squared: 0.1551
Epoch 241/500
482/482 [==============================] - 1s 2ms/step - loss: 45.5656 - r_squared: 0.1046
Epoch 242/500
482/482 [==============================] - 1s 2ms/step - loss: 48.2099 - r_squared: 0.1235
Epoch 243/500
482/482 [==============================] - 1s 2ms/step 

482/482 [==============================] - 1s 2ms/step - loss: 47.8198 - r_squared: 0.1920
Epoch 313/500
482/482 [==============================] - 1s 2ms/step - loss: 47.3754 - r_squared: -0.0609
Epoch 314/500
482/482 [==============================] - 1s 1ms/step - loss: 47.9379 - r_squared: 0.1174
Epoch 315/500
482/482 [==============================] - 1s 1ms/step - loss: 47.1669 - r_squared: 0.2020
Epoch 316/500
482/482 [==============================] - 1s 2ms/step - loss: 46.5774 - r_squared: 0.1946
Epoch 317/500
482/482 [==============================] - 1s 2ms/step - loss: 47.0693 - r_squared: 0.1075TA: 0s - loss: 43.2237
Epoch 318/500
482/482 [==============================] - 1s 2ms/step - loss: 47.3680 - r_squared: 0.1230
Epoch 319/500
482/482 [==============================] - 1s 2ms/step - loss: 47.8097 - r_squared: 0.0654
Epoch 320/500
482/482 [==============================] - 1s 1ms/step - loss: 45.3420 - r_squared: 0.1723
Epoch 321/500
482/482 [=======================

482/482 [==============================] - 1s 2ms/step - loss: 47.3547 - r_squared: 0.1184
Epoch 390/500
482/482 [==============================] - 1s 2ms/step - loss: 47.8152 - r_squared: 0.1497
Epoch 391/500
482/482 [==============================] - 1s 2ms/step - loss: 48.0487 - r_squared: 0.2048
Epoch 392/500
482/482 [==============================] - 1s 2ms/step - loss: 44.5440 - r_squared: 0.1430
Epoch 393/500
482/482 [==============================] - 1s 1ms/step - loss: 47.2334 - r_squared: 0.1099
Epoch 394/500
482/482 [==============================] - 1s 2ms/step - loss: 48.5765 - r_squared: 0.1199
Epoch 395/500
482/482 [==============================] - 1s 1ms/step - loss: 46.7458 - r_squared: 0.1325
Epoch 396/500
482/482 [==============================] - 1s 2ms/step - loss: 46.6117 - r_squared: 0.1470
Epoch 397/500
482/482 [==============================] - 1s 2ms/step - loss: 45.7749 - r_squared: 0.1770
Epoch 398/500
482/482 [==============================] - 1s 1ms/step 

482/482 [==============================] - 1s 2ms/step - loss: 47.1583 - r_squared: 0.1277
Epoch 468/500
482/482 [==============================] - 1s 2ms/step - loss: 46.3588 - r_squared: 0.2022
Epoch 469/500
482/482 [==============================] - 1s 2ms/step - loss: 45.9170 - r_squared: 0.1303
Epoch 470/500
482/482 [==============================] - 1s 2ms/step - loss: 47.9806 - r_squared: 0.1891
Epoch 471/500
482/482 [==============================] - 1s 2ms/step - loss: 47.2209 - r_squared: 0.1794
Epoch 472/500
482/482 [==============================] - 1s 2ms/step - loss: 45.7360 - r_squared: 0.1549
Epoch 473/500
482/482 [==============================] - 1s 2ms/step - loss: 46.8791 - r_squared: 0.1370
Epoch 474/500
482/482 [==============================] - 1s 2ms/step - loss: 46.6688 - r_squared: 0.1232
Epoch 475/500
482/482 [==============================] - 1s 2ms/step - loss: 46.9943 - r_squared: 0.1692
Epoch 476/500
482/482 [==============================] - 1s 2ms/step 

VotingRegressor(estimators=[('random-forest',
                             RandomForestRegressor(n_estimators=500)),
                            ('mlp',
                             KerasRegressor(batch_size=200, dropout=0.5, epochs=500, hidden_layer_sizes=(100,), loss='mse', metrics=[<function KerasRegressor.r_squared at 0x000002549CDB8E50>], model=<function get_reg at 0x000002549D1464C0>)),
                            ('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          colsamp...
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=0.1,
                                          max_delta_step=None, max_depth=5,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
       

In [13]:
ensemble_model_y2.fit(X_train2, y2_train)

Epoch 1/600
1926/1926 [==============================] - 4s 2ms/step - loss: 32743.6836 - r_squared: 0.0169
Epoch 2/600
1926/1926 [==============================] - 3s 1ms/step - loss: 25892.5098 - r_squared: 0.2114
Epoch 3/600
1926/1926 [==============================] - 3s 1ms/step - loss: 24421.4590 - r_squared: 0.2516
Epoch 4/600
1926/1926 [==============================] - 2s 1ms/step - loss: 23935.0566 - r_squared: 0.2633
Epoch 5/600
1926/1926 [==============================] - 2s 1ms/step - loss: 23690.5918 - r_squared: 0.2741
Epoch 6/600
1926/1926 [==============================] - 3s 1ms/step - loss: 23484.4238 - r_squared: 0.2824
Epoch 7/600
1926/1926 [==============================] - 3s 2ms/step - loss: 23357.1543 - r_squared: 0.2861
Epoch 8/600
1926/1926 [==============================] - 3s 1ms/step - loss: 23333.9570 - r_squared: 0.2783
Epoch 9/600
1926/1926 [==============================] - 3s 1ms/step - loss: 23295.9238 - r_squared: 0.2814
Epoch 10/600
1926/1926 [====

1926/1926 [==============================] - 3s 1ms/step - loss: 21800.3379 - r_squared: 0.3272
Epoch 74/600
1926/1926 [==============================] - 2s 1ms/step - loss: 21771.4551 - r_squared: 0.3311
Epoch 75/600
1926/1926 [==============================] - 2s 1ms/step - loss: 21791.1934 - r_squared: 0.3241
Epoch 76/600
1926/1926 [==============================] - 2s 1ms/step - loss: 21738.4395 - r_squared: 0.3311
Epoch 77/600
1926/1926 [==============================] - 2s 1ms/step - loss: 21651.3301 - r_squared: 0.3256
Epoch 78/600
1926/1926 [==============================] - 2s 1ms/step - loss: 21676.1895 - r_squared: 0.3335
Epoch 79/600
1926/1926 [==============================] - 3s 2ms/step - loss: 21748.1074 - r_squared: 0.3301
Epoch 80/600
1926/1926 [==============================] - 2s 1ms/step - loss: 21754.6172 - r_squared: 0.3301
Epoch 81/600
1926/1926 [==============================] - 2s 1ms/step - loss: 21639.4375 - r_squared: 0.3350
Epoch 82/600
1926/1926 [========

1926/1926 [==============================] - 2s 1ms/step - loss: 21017.3809 - r_squared: 0.3491
Epoch 148/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20960.5293 - r_squared: 0.3541
Epoch 149/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20834.9844 - r_squared: 0.3580
Epoch 150/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20829.0586 - r_squared: 0.3602
Epoch 151/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20903.8281 - r_squared: 0.3531
Epoch 152/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20896.3105 - r_squared: 0.3587
Epoch 153/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20885.5000 - r_squared: 0.3540
Epoch 154/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20881.7578 - r_squared: 0.3573
Epoch 155/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20795.7441 - r_squared: 0.3559
Epoch 156/600
1926/1926 

1926/1926 [==============================] - 2s 1ms/step - loss: 20256.6504 - r_squared: 0.3740
Epoch 222/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20399.9766 - r_squared: 0.3703
Epoch 223/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20238.5547 - r_squared: 0.3780
Epoch 224/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20285.4434 - r_squared: 0.3760
Epoch 225/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20267.3887 - r_squared: 0.3686
Epoch 226/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20341.0586 - r_squared: 0.3676
Epoch 227/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20372.6934 - r_squared: 0.3703
Epoch 228/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20299.0195 - r_squared: 0.3760
Epoch 229/600
1926/1926 [==============================] - 3s 1ms/step - loss: 20148.1660 - r_squared: 0.3797
Epoch 230/600
1926/1926 

1926/1926 [==============================] - 2s 1ms/step - loss: 20134.4277 - r_squared: 0.3770
Epoch 296/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20076.4492 - r_squared: 0.3825
Epoch 297/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20047.9102 - r_squared: 0.3782
Epoch 298/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20150.5566 - r_squared: 0.3742
Epoch 299/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20099.6309 - r_squared: 0.3767
Epoch 300/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20056.8398 - r_squared: 0.3780
Epoch 301/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20148.7324 - r_squared: 0.3757
Epoch 302/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20064.1973 - r_squared: 0.3821
Epoch 303/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20084.9453 - r_squared: 0.3800
Epoch 304/600
1926/1926 

1926/1926 [==============================] - 2s 1ms/step - loss: 20020.2148 - r_squared: 0.3814
Epoch 370/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20075.6348 - r_squared: 0.3801
Epoch 371/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19851.5664 - r_squared: 0.3895A: 2s 
Epoch 372/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19957.9824 - r_squared: 0.3846
Epoch 373/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19905.9648 - r_squared: 0.3831
Epoch 374/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20108.4492 - r_squared: 0.3748
Epoch 375/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19956.9883 - r_squared: 0.3820
Epoch 376/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20001.4902 - r_squared: 0.3822
Epoch 377/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20047.3145 - r_squared: 0.3803
Epoch 378/600
1926

1926/1926 [==============================] - 2s 1ms/step - loss: 19916.7383 - r_squared: 0.3847
Epoch 444/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20099.0234 - r_squared: 0.3790
Epoch 445/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19928.8770 - r_squared: 0.3852
Epoch 446/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19972.3848 - r_squared: 0.3813
Epoch 447/600
1926/1926 [==============================] - 2s 1ms/step - loss: 20007.9258 - r_squared: 0.3841
Epoch 448/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19811.3418 - r_squared: 0.3894
Epoch 449/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19837.3340 - r_squared: 0.3874
Epoch 450/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19850.0547 - r_squared: 0.3831
Epoch 451/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19910.3184 - r_squared: 0.3792
Epoch 452/600
1926/1926 

1926/1926 [==============================] - 2s 1ms/step - loss: 19969.0547 - r_squared: 0.3847
Epoch 518/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19897.3535 - r_squared: 0.3832
Epoch 519/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19850.9043 - r_squared: 0.3928
Epoch 520/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19914.0762 - r_squared: 0.3855
Epoch 521/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19872.1621 - r_squared: 0.3849
Epoch 522/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19882.8867 - r_squared: 0.3878
Epoch 523/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19809.1270 - r_squared: 0.3895
Epoch 524/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19895.0352 - r_squared: 0.3880
Epoch 525/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19841.6465 - r_squared: 0.3852
Epoch 526/600
1926/1926 

1926/1926 [==============================] - 2s 1ms/step - loss: 19890.9453 - r_squared: 0.3809
Epoch 592/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19811.4551 - r_squared: 0.3876
Epoch 593/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19877.1348 - r_squared: 0.3887
Epoch 594/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19833.0938 - r_squared: 0.3880
Epoch 595/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19837.7754 - r_squared: 0.3905
Epoch 596/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19909.9648 - r_squared: 0.3846
Epoch 597/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19812.4590 - r_squared: 0.3865
Epoch 598/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19905.5801 - r_squared: 0.3852
Epoch 599/600
1926/1926 [==============================] - 2s 1ms/step - loss: 19892.5898 - r_squared: 0.3865
Epoch 600/600
1926/1926 

VotingRegressor(estimators=[('random-forest',
                             RandomForestRegressor(n_estimators=1000)),
                            ('mlp',
                             KerasRegressor(batch_size=50, dropout=0.5, epochs=600, hidden_layer_sizes=(100,), loss='mse', metrics=[<function KerasRegressor.r_squared at 0x000002549CDB8E50>], model=<function get_reg at 0x000002549D1464C0>)),
                            ('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          colsamp...
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=0.1,
                                          max_delta_step=None, max_depth=6,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
       

In [15]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [16]:
print('Train set evaluation Y1:\n_____________________________________')
print_evaluate(y1_train, ensemble_model_y1.predict(X_train1))

Train set evaluation Y1:
_____________________________________
482/482 [==============================] - 0s 597us/step
MAE: 0.9461894717876128
MSE: 14.005275338171362
RMSE: 3.742362267094323
R2 Square 0.8033506756173131
__________________________________


In [18]:
print('Train set evaluation Y2:\n_____________________________________')
print_evaluate(y2_train, ensemble_model_y2.predict(X_train2))

Train set evaluation Y2:
_____________________________________
1926/1926 [==============================] - 1s 612us/step
MAE: 56.61943030644907
MSE: 9180.279272981403
RMSE: 95.81377392098383
R2 Square 0.7325900531845706
__________________________________


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y2_train, ensemble_model_y2.predict(X_train1)))

In [26]:
Y1_pred =ensemble_model_y1.predict(X_test1)
Y2_pred = ensemble_model_y2.predict(X_test2)

KeyboardInterrupt: 

In [20]:
Y1_pred = Y1_pred.tolist()

In [21]:
Y2_pred = Y2_pred.tolist()

In [23]:
submission = pd.DataFrame({
        "UNIQUE_IDENTIFIER": df_test["UNIQUE_IDENTIFIER"],
        "Y1": Y1_pred,
        "Y2": Y2_pred
    })

In [24]:
df2 = submission.groupby(['UNIQUE_IDENTIFIER'],as_index = False).agg('mean')

In [25]:
df2.to_csv('./submission_final.csv', index=False)

In [52]:
df_test

,Unnamed: 0,UNIQUE_IDENTIFIER,STATUS_CHECK,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,...,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,CATEGORY_1_0,CATEGORY_1_1,CATEGORY_1_2,CATEGORY_2_0,CATEGORY_2_1,CATEGORY_2_2
0,0,98481269471,0,0.059750,0.009050,0.004100,0.000000,0.416000,0.055000,0.003000,...,0.000000,0.00000,0.000000,0.00000,1,0,0,0,0,1
1,0,98481270499,1,1.733000,0.259950,1.010650,0.000000,0.000000,0.010000,0.001000,...,0.000000,0.00000,0.000000,0.00000,1,0,0,0,0,1
2,0,98481272694,0,0.000132,0.000020,0.000045,0.000000,0.004211,0.010526,0.000105,...,2.775126,1.68469,1.419848,0.42207,1,0,0,1,0,0
3,0,98481273365,0,0.649906,0.099128,0.332084,0.298118,0.106994,0.262707,0.017960,...,0.000000,0.00000,0.000000,0.00000,1,0,0,0,0,1
4,0,98481278321,0,0.785288,0.078207,0.140675,0.000000,0.458000,0.402000,0.050200,...,0.000000,0.00000,0.000000,0.00000,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65237,0,98537204815,1,0.903473,0.126131,0.649199,0.136504,0.404057,0.101460,0.020292,...,0.000000,0.00000,0.000000,0.00000,0,0,1,0,0,1
65238,0,98537204877,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.100000,...,0.000000,0.00000,0.000000,0.00000,0,1,0,1,0,0
65239,0,98537204881,0,0.158675,0.024084,0.047972,0.019489,0.089760,0.082455,0.016582,...,0.000000,0.00000,0.000000,0.00000,1,0,0,0,1,0
65240,0,98537205013,1,0.005124,0.001025,0.000639,0.000000,0.000000,0.000208,0.000083,...,0.000000,0.00000,0.000000,0.00000,1,0,0,1,0,0


In [78]:
X_train1

,STATUS_CHECK,ENTRY,REVENUE,WINNINGS_1,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,WITHDRAW,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,CATEGORY_1_0,CATEGORY_1_1,CATEGORY_1_2,CATEGORY_2_0,CATEGORY_2_1,CATEGORY_2_2
0,0,0.098334,0.007531,0.043399,0.000000,0.000714,0.000286,0.000000,0.000000,0.891238,0.377524,1,0,0,1,0,0
1,1,31.392245,3.803991,25.940547,0.866865,11.122807,0.050339,1.568795,0.397661,0.527497,0.165988,1,0,0,0,1,0
2,0,0.018567,0.001624,0.010514,0.005791,0.000278,0.000111,0.000000,0.000556,0.280510,0.055431,1,0,0,0,0,1
3,0,0.747600,0.117320,0.025330,0.240000,0.504000,0.020400,0.000000,0.500800,0.919600,0.059600,1,0,0,1,0,0
4,0,0.500000,0.080000,0.000000,0.000000,0.500000,0.100000,0.000000,0.000000,0.200000,0.000000,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96293,0,1.267884,0.189529,0.589838,0.578000,0.384167,0.035167,0.000000,0.000000,0.482167,0.128833,0,0,1,1,0,0
96294,0,47.909122,6.131328,40.061926,1.735889,4.858611,0.119500,0.000000,1.341389,10.365611,3.137167,1,0,0,0,1,0
96295,0,1.115944,0.148308,0.669854,0.038648,0.371466,0.063021,0.000000,0.004572,1.194554,0.245474,0,0,1,0,1,0
96296,0,0.381386,0.057377,0.361288,0.013333,0.000167,0.000067,0.000000,0.000000,0.802166,0.338329,0,0,1,0,0,1


In [90]:
X_test1

,STATUS_CHECK,ENTRY,REVENUE,WINNINGS_1,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,WITHDRAW,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,CATEGORY_1_0,CATEGORY_1_1,CATEGORY_1_2,CATEGORY_2_0,CATEGORY_2_1,CATEGORY_2_2
0,0,0.059750,0.009050,0.004100,0.416000,0.055000,0.003000,0.000000,0.098000,0.014000,0.002000,1,0,0,0,0,1
1,1,1.733000,0.259950,1.010650,0.000000,0.010000,0.001000,0.000000,0.000000,3.466000,1.189000,1,0,0,0,0,1
2,0,0.000132,0.000020,0.000045,0.004211,0.010526,0.000105,0.000000,0.000000,0.000526,0.000105,1,0,0,1,0,0
3,0,0.649906,0.099128,0.332084,0.106994,0.262707,0.017960,0.000000,0.034450,0.394245,0.077140,1,0,0,0,0,1
4,0,0.785288,0.078207,0.140675,0.458000,0.402000,0.050200,0.000000,0.000000,4.999800,1.768800,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65237,1,0.903473,0.126131,0.649199,0.404057,0.101460,0.020292,0.005674,0.005540,4.188752,0.859331,0,0,1,0,0,1
65238,0,0.000000,0.000000,0.000000,0.000000,0.250000,0.100000,0.000000,0.050000,0.000000,0.000000,0,1,0,1,0,0
65239,0,0.158675,0.024084,0.047972,0.089760,0.082455,0.016582,0.000000,0.028182,0.341018,0.063273,1,0,0,0,1,0
65240,1,0.005124,0.001025,0.000639,0.000000,0.000208,0.000083,0.000000,0.004344,0.053529,0.001504,1,0,0,1,0,0


In [93]:
train=pd.read_csv("./train.csv",delimiter="\t")

In [122]:
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
UNIQUE_IDENTIFIER,1246837.0,9.852168e+10,6.339280e+06,9.848127e+10,9.851986e+10,9.852279e+10,9.852542e+10,9.852732e+10
SEQUENCE_NO,1246837.0,1.105455e+01,7.850787e+00,1.000000e+00,4.000000e+00,1.000000e+01,1.700000e+01,3.000000e+01
STATUS_CHECK,1246837.0,4.223696e-01,4.939370e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
ACTIVE_YN,1246837.0,9.868451e-01,1.139379e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
ENTRY,1246837.0,4.726720e+00,2.609701e+01,0.000000e+00,0.000000e+00,4.500000e-01,1.962600e+00,2.819799e+03
REVENUE,1246837.0,5.350111e-01,2.557525e+00,0.000000e+00,0.000000e+00,5.970000e-02,2.637500e-01,2.096527e+02
WINNINGS_1,1246837.0,3.700804e+00,2.222386e+01,0.000000e+00,0.000000e+00,1.888000e-01,1.276900e+00,2.086580e+03
WINNINGS_2,1246837.0,4.262171e-02,8.355851e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.531417e+02
DISCOUNT,1246837.0,4.482682e-01,2.193891e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+02
DEPOSIT,1246837.0,7.602949e-01,3.407900e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e-01,1.250000e+02


In [119]:
train['CATEGORY_2'].unique()

array(['B', 'I', 'D', 'E', 'A', 'C', 'F', 'H', 'G'], dtype=object)

In [146]:
from ipywidgets import interact, widgets


def myfunc(x,y,z,a,b,c,d,e,f,g,h,i,j):
    mlist=[]
    mlist.append(x)
    mlist.append(y)
    mlist.append(z)
    mlist.append(a)
    mlist.append(b)
    mlist.append(c)
    mlist.append(d)
    mlist.append(e)
    mlist.append(f)
    mlist.append(g)
    mlist.append(h)
    mlist.append(i)
    mlist.append(j)
    dfde=pd.DataFrame([mlist],columns=['STATUS_CHECK','ENTRY','REVENUE','WINNINGS_1','DISCOUNT','DEPOSIT','DEPOSIT_NUMBER','WITHDRAW','DEPOSIT_TRAILS','ENTRY_NUMBER','WINNINGS_NUMBER','CATEGORY_1','CATEGORY_2'])
    dd=encode_in(dfde)
    

    return dd

interact(myfunc,x=widgets.Dropdown(options=['0','1'],value='0',description='STATUS CHECK:',disabled=False,),
         y=widgets.FloatSlider(min=0,max=2.819799e+03,step=0.001,value=0,description='ENTRY:', orientation='horizontal',readout_format='.1f'),
         z=widgets.FloatSlider(min=0,max=2.096527e+02,step=0.001,value=0,description='REVENUE:', orientation='horizontal',readout_format='.1f'),
        a=widgets.FloatSlider(min=0,max=2.086580e+03,step=0.001,value=0,description='WINNINGS 1:', orientation='horizontal',readout_format='.1f'),
        b=widgets.FloatSlider(min=0,max=2.000000e+02,step=0.001,value=0,description='DISCOUNT:', orientation='horizontal',readout_format='.1f'),
        c=widgets.FloatSlider(min=0,max=1.250000e+02,step=0.001,value=0,description='DEPOSIT:', orientation='horizontal',readout_format='.1f'),
        d=widgets.FloatSlider(min=0,max=3.000000e+00,step=0.001,value=0,description='DEPOSIT_NUMBER:', orientation='horizontal',readout_format='.1f'),
        e=widgets.FloatSlider(min=0,max=2.400000e+02,step=0.001,value=0,description='WITHDRAW:', orientation='horizontal',readout_format='.1f'),
        f=widgets.FloatSlider(min=0,max=6.450000e+02,step=0.001,value=0,description='DEPOSIT_TRAILS:', orientation='horizontal',readout_format='.1f'),
        g=widgets.FloatSlider(min=0,max=1.077000e+02,step=0.001,value=0,description='ENTRY_NUMBER:', orientation='horizontal',readout_format='.1f'),
        h=widgets.FloatSlider(min=0,max=4.510000e+01,step=0.001,value=0,description='WINNINGS_NUMBER:', orientation='horizontal',readout_format='.1f'),
        i=widgets.Dropdown(options=['M', 'C', 'I', 'B', 'K', 'G', 'A', 'L', 'J', 'F'],description='CATEGORY 1:',disabled=False,),
        j=widgets.Dropdown(options=['B', 'I', 'D', 'E', 'A', 'C', 'F', 'H', 'G'],description='CATEGORY 2:',disabled=False,))


interactive(children=(Dropdown(description='STATUS CHECK:', options=('0', '1'), value='0'), FloatSlider(value=…

<function __main__.myfunc(x, y, z, a, b, c, d, e, f, g, h, i, j)>